# Texas Tow Trucks

We're going to scrape some [tow trucks in Texas](https://www.tdlr.texas.gov/tools_search/). 

# Part One: Building a company list

Search for businesses with the word **WRECK** in their names.

* **Tip:** Start by scraping the first page to a dataframe, then expand to a loop that combines all of the pages. Finally combine all of the dataframes with `pd.concat`. You might find [this page helpful](https://jonathansoma.com/everything/scraping/pagination/), although the "On an interactive site" example uses Selenium instead of Playwright. You should be able to figure out how to change it!
* **Tip:** You can't just do a `try`/`except`, because even if you ask for page 99999 it will always give you the last page again! Watch out that you don't get stuck in an infinite loop!

In [77]:
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
import csv

In [59]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=False)

In [60]:
page = await browser.new_page()

In [61]:
await page.goto("https://www.tdlr.texas.gov/tools_search/")

<Response url='https://www.tdlr.texas.gov/tools_search/' request=<Request url='https://www.tdlr.texas.gov/tools_search/' method='GET'>>

In [62]:
await page.locator("#namebutton").click()

In [63]:
await page.locator("#namedata").fill("WRECK")

In [64]:
await page.locator("#submit3").click()

In [68]:
doc = BeautifulSoup(await page.content(), 'html.parser')

In [85]:
doc.select("table")

[<table border="0" cellpadding="0" cellspacing="0" width="88%">
 <tbody><tr>
 <td align="LEFT" style="width: 64%"><font face="Arial" size="3"><strong> Tow Company and Vehicle Facility Storage Inquiry Results</strong></font></td>
 <td align="RIGHT" width="36%"><font face="Arial" size="2">Total number of records : 802</font></td>
 </tr>
 </tbody></table>,
 <table border="0" cellpadding="0" cellspacing="0" width="88%">
 <tbody><tr>
 <td align="LEFT" width="50%"><font face="Arial" size="2">Page <strong>1</strong> of  <strong>41</strong></font></td>
 <td align="RIGHT" width="50%"><font face="Arial" size="2"><a href="mccs_search_process.asp?ls_search_print=TRUE&amp;stype=name&amp;ls_temp=
 		WRECK&amp;ls_carrier_type=COMPANY&amp;ls_cert_status=">
 <img alt="Remove Breakdown" border="0" src="ico-printer_remove.gif"/></a></font></td>
 </tr>
 </tbody></table>,
 <table align="center" border="2" bordercolor="black" cellpadding="1" cellspacing="1" width="88%">
 <tbody><tr>
 <td align="left" bgcolo

In [83]:
import pandas as pd

In [90]:
table = doc.find('table', align='center', width='88%')
headers = [header.text.strip() for header in table.find_all('td')]
data_rows = table.find_all('tr')[1:]

for row in data_rows:
    cells = row.find_all('td')
    customers.append(cells[0].text.strip())
    dba_names.append(cells[1].text.strip())
    tdlr_numbers.append(cells[2].text.strip())
    cities.append(cells[3].text.strip())
    states.append(cells[4].text.strip())
    zip_codes.append(cells[5].text.strip())

data = {
    'Customer': customers,
    'DBA Name': dba_names,
    'TDLR Number': tdlr_numbers,
    'City': cities,
    'State': states,
    'Zip code': zip_codes
}

df = pd.DataFrame(data)
print(df)

                          Customer                       DBA Name  \
0      CALIBER WRECKER SERVICE LLC                                  
1       1ST CHOICE WRECKER SERVICE   1ST CHOICE PAINT & BODY, INC   
2       1ST CHOICE WRECKER SERVICE  1ST CHOICE PAINT & BODY, INC.   
3   1ST CHOICE WRECKER SERVICE LLC                                  
4   1ST CHOICE WRECKER SERVICE LLC                                  
5   1ST CHOICE WRECKER SERVICE LLC                                  
6       1STCHOICEWRECKERSERVICELLC                                  
7     24 HOUR WRECKER SERVICE INC.                                  
8     24 HOUR WRECKER SERVICE, INC                                  
9         24/7 WRECKER SERVICE LLC                                  
10        24/7 WRECKER SERVICE LLC                                  
11         290 WRECKER SERVICE INC                HWY 290 WRECKER   
12        290 WRECKER SERVICE INC.                HWY 290 WRECKER   
13       290 WRECKER SERVICE, INC.

### Cleanup

If you haven't already, rename the columns to be:
    
    * Customer
    * DBA Name
    * TDLR Number
    * City
    * State
    * Zip code

and remove all of the rows where the customer name is `Customer`.

In [ ]:
#Why would the customer name be customer, if we searched with the name "wreck"?

## Save as `wreckers.csv`

In [86]:
df.to_csv('wreckers.csv', index=False)

In [88]:
!ls

00 - Scraping basics for Playwright.ipynb
01 - Scraping - Texas Cosmetology Violations.ipynb
02 - Tow Trucks.ipynb
headlines.csv
trucks-subset.csv
wreck.csv
wreckers.csv


# Part Two: Company info

> You can use whatever tool you'd like for this, but form submission doesn't necessarily mean Playwright! If you want to go the `requests` route instead, feel free to [look at this page](https://jonathansoma.com/everything/scraping/pretending-to-be-a-browser/) about several ways to pretend to be a browser.

## Step 1: Scraping one page

Try searching from the [tools page](https://www.tdlr.texas.gov/tools_search/) for the TDLR Number `006556161C`. From the results page, scrape the:

* Business name
* Phone number
* License status
* Physical address

And save the results into a dictionary. Also include `TDLR Number` in the dictionary. **Print the dictionary**.

> ***Tip:** It's best if each item has its own key, but **it's fine to pull "larger" sections of the page and split them up in pandas later on***
>
> ***Tip:** Be sure you don't forget to include `TDLR Number` in the dictionary!*

In [93]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=False)

In [94]:
page2 = await browser.new_page()

In [96]:
await page2.goto("https://www.tdlr.texas.gov/tools_search/")

<Response url='https://www.tdlr.texas.gov/tools_search/' request=<Request url='https://www.tdlr.texas.gov/tools_search/' method='GET'>>

In [103]:
await page2.locator("#mcrbutton").click()

In [104]:
await page2.locator("#mcrdata").fill("006556161C")

In [105]:
await page2.locator("#submit3").click()

In [101]:
doc2 = BeautifulSoup(await page2.content(), 'html.parser')

In [106]:
doc2.select("table")

[<table align="center" width="85%">
 <tbody><tr>
 <td align="center" valign="top" width="60%">
 <font face="Arial" size="3"><b>TDLR Tow Company and Vehicle Storage Facility Inquiry Page</b></font>
 </td>
 </tr>
 <tr>
 <td style="width:20%; text-align:center"><a href="ConsortiumReport.asp">TDLR Approved Consortium List</a></td>
 </tr>
 </tbody></table>,
 <table align="center" border="1" width="85%">
 <tbody><tr>
 <td bgcolor="#oo33ff" width="180"><center><b><font color="white">Search By:</font></b></center></td>
 <td bgcolor="#oo33ff"><center><b><font color="white">Description</font></b></center></td>
 </tr>
 <tr>
 <td bgcolor="#99CCFF">
 </td>
 <td bgcolor="#99CCFF"> </td>
 </tr>
 <tr>
 <td>
 <input id="namebutton" name="searchtype" onclick="//document.searchinfo.namedata.style.visibility='visible';
 			    searchinfo.namedata.focus();
 			    searchinfo.name_carrier_type.style.visibility='visible';			
 			    //document.searchinfo.mcrdata.value='';
 			    //document.searchinfo.mcrdat

## Step 2: Move into one cell

Move the code above all into one cell that relies on the variable `tdlr_number`.

Add the code below to the page and confirm that it displays the data for the correct result.

```python
tdlr_number = '0654479VSF'
```

Confirm that the information is correct. Did it not work out? Go back and edit your selectors, or be a little broader in the parts of the page you sweep up.

## Step 3: Scraping many pages

Using pandas, read in `trucks-subset.csv`.

## Scrape every single row, creating a list of dictionaries from the scraped data.

You should never use for loops with pandas *except when working with Playwright*. To use loops in pandas you'll make use of `.iterrows()`.

The code below loops through a dataframe called `df` and prints out the `address` column.

```python
for index, row in df.iterrows():
    print(row['address'])
```

You'll adapt this code to use your dataframe, and combine it with the scraping code you wrote above.

> ***Tip:** This is like what we did for the townships in class*

## Save your dataframe as `data-uncleaned.csv`

# Cleaning your data

## Re-open the `data-uncleaned.csv` file

You probably want to set `pd.options.display.max_colwidth`

## Clean it up!

Make sure there are columns for

- Business name
- Phone number
- License status
- Physical address

And drop all of the other columns (The easiest way is to use `df = df.drop(columns=[...])`)

## Combine with the original CSV file

To combine dataframes based on indexes, you use `df.join(other_df)`. If you'd prefer to match by columns, you could also use `df.merge(other_df, left_on='...', right_on='...')` and tell it the two columns that match between the two.

### Save to a CSV